In [ ]:
!pip install spacy-langdetect
!pip install language-detector
!pip install symspellpy
!pip install sentence-transformers

In [ ]:
# conda install tensorflow==1.12.0
!pip install tensorflow --upgrade

In [ ]:
import os
import json
import pandas as pd
import numpy as np
from nltk.corpus import wordnet
import re
import matplotlib.pyplot as plt
from tqdm import tqdm
from nltk.corpus import stopwords


In [ ]:
import os
import numpy as np
import pandas as pd
import gensim
from gensim.parsing.preprocessing import STOPWORDS
from gensim.utils import simple_preprocess
from gensim import corpora, models
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import datetime
import time
from mpl_toolkits.mplot3d import Axes3D

%matplotlib inline

# Load library
import matplotlib.pyplot as plt


In [ ]:
train_data = pd.read_csv("/content/sample_data/Train.csv")
print(train_data.shape)

(14004, 31)


In [ ]:
count = 0
indices = []

length = len(train_data)

for i in range(length):
    if type(train_data.iloc[i, 3])== float:
        count += 1
    else:
        indices.append(i)

In [ ]:
##extract the abstract to pandas
documents = train_data.iloc[indices, 1]
documents=documents.reset_index()
documents.drop("index", inplace = True, axis = 1)

##create pandas data frame with all abstracts, use as input corpus
documents["index"] = documents.index.values
documents.head(3)

,ABSTRACT,index
0,a ever-growing datasets inside observational a...,0
1,we propose the framework considering optimal $...,1
2,nanostructures with open shell transition meta...,2


In [ ]:
# documents[0]

In [ ]:
pip install umap_learn

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score
from collections import Counter
import umap
from wordcloud import WordCloud
from gensim.models.coherencemodel import CoherenceModel



---


**Organizes the tokenized documents into topics based on their cluster labels and identifies the most frequent words within each topic.**



---



In [ ]:
def get_topic_words(token_lists, labels, k=None):
    """
    get top words within each topic from clustering results
    """
    if k is None:
        k = len(np.unique(labels))
    topics = ['' for _ in range(k)]
    for i, c in enumerate(token_lists):
        topics[labels[i]] += (' ' + ' '.join(c))
    word_counts = list(map(lambda x: Counter(x.split()).items(), topics))
    # get sorted word counts
    word_counts = list(map(lambda x: sorted(x, key=lambda x: x[1], reverse=True), word_counts))
    # get topics
    topics = list(map(lambda x: list(map(lambda x: x[0], x[:10])), word_counts))

    return topics

In [ ]:
def get_top_words_per_topic(lists_tokens, cluster_labels, k):
  topics = []
  print(f"K Values: {k}")

  for k_val in range(k):
    topics.append('')

  for i, tokens in enumerate(lists_tokens):
    topics[cluster_labels[i]] += (' ' + ' '.join(tokens))

    topic_word_counts = list(
        map(
            lambda topic: Counter(topic.split()).items(),
            topics
        )
    )

    print(f"Topic word counts : {topic_word_counts}")

    sorted_word_counts = list(
        map(
            lambda topic_counts: sorted(topic_counts, key=lambda topic_counts: topic_counts[1], reverse=True),
            topic_word_counts
        )
    )

    print(f"Sorted word counts : {sorted_word_counts}")

    top_words_per_topic = list(
        map(
            lambda topic_counts: list(map(lambda topic_counts: topic_counts[0], topic_counts[:10])),
            sorted_word_counts
        )
    )

    print(f"Top Words Per Topic : {top_words_per_topic}")

    return top_words_per_topic



---


**The get_coherence function calculates the coherence score for a given topic model. The coherence score is a measure of how coherent or interpretable the topics generated by the model are.**


---



In [ ]:
def get_coherence(model, token_lists, measure='c_v'):
    if model.method == 'LDA':
        cm = CoherenceModel(model=model.ldamodel, texts=token_lists, corpus=model.corpus, dictionary=model.dictionary,
                            coherence=measure)
    else:
        topics = get_topic_words(token_lists, model.cluster_model.labels_)
        cm = CoherenceModel(topics=topics, texts=token_lists, corpus=model.corpus, dictionary=model.dictionary,
                            coherence=measure)
    return cm.get_coherence()



---
**Silhouette Score**


---



In [ ]:
def get_silhouette(model):
    # if model.method == 'LDA':
    #     return
    lbs = model.cluster_model.labels_
    vec = model.vec[model.method]
    return silhouette_score(vec, lbs)

In [ ]:
def plot_proj(embedding, lbs):
    n = len(embedding)
    counter = Counter(lbs)
    for i in range(len(np.unique(lbs))):
        plt.plot(embedding[:, 0][lbs == i], embedding[:, 1][lbs == i], '.', alpha=0.5,
                 label='cluster {}: {:.2f}%'.format(i, counter[i] / n * 100))
    plt.legend(loc = 'best')
    plt.grid(color ='grey', linestyle='-',linewidth = 0.25)

In [ ]:
def get_wordcloud(model, token_lists, topic):
    # if model.method == 'LDA':
    #     return
    print('Getting wordcloud for topic {}'.format(topic))
    lbs = model.cluster_model.labels_
    tokens = ' '.join([' '.join(_) for _ in np.array(token_lists)[lbs == topic]])

    wordcloud = WordCloud(width=800, height=560,
                          background_color='white', collocations=False,
                          min_font_size=10).generate(tokens)

    # plot the WordCloud image
    plt.figure(figsize=(8, 5.6), facecolor=None)
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.tight_layout(pad=0)
    dr = '/kaggle/working/{}/{}'.format(model.method, model.id)
    if not os.path.exists(dr):
        os.makedirs(dr)
    plt.savefig('/kaggle/working' + '/Topic' + str(topic) + '_wordcloud')
    print('Getting wordcloud for topic {}. Done!'.format(topic))

In [ ]:
!pip install stop-words
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
import re
import nltk
from nltk.tokenize import word_tokenize
from language_detector import detect_language

import pkg_resources
from symspellpy import SymSpell, Verbosity

sym_spell = SymSpell(max_dictionary_edit_distance=3, prefix_length=7)
dictionary_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_dictionary_en_82_765.txt")
if sym_spell.word_count:
    pass
else:
    sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)




---

**Preprocessing sentences**

---



In [ ]:
import re

def preprocess_text(input_text):
  processed_text = re.sub(r'([a-z])([A-Z])', r'\1\. \2', input_text)
  processed_text = processed_text.lower()
  processed_text = re.sub(r'&gt|&lt', ' ', processed_text)
  processed_text = re.sub(r'([a-z])\1{2,}', r'\1', processed_text)
  processed_text = re.sub(r'([\W+])\1{1,}', r'\1', processed_text)
  processed_text = re.sub(r'\*|\W\*|\*\W', '. ', processed_text)
  processed_text = re.sub(r'\(.*?\)', '. ', processed_text)
  processed_text = re.sub(r'\W+?\.', '.', processed_text)
  processed_text = re.sub(r' ing ', ' ', processed_text)
  processed_text = re.sub(r'(.{2,}?)\1{1,}', r'\1', processed_text)
  processed_text = re.sub(r'product received for free[.| ]', ' ', processed_text)
  processed_text = re.sub(r'(\.|\?|!)(\w)', r'\1 \2', processed_text)

  return processed_text.strip()


def preprocess_sentence(abstract_text):
  print(f"In Preprocess_Sentence")
  preprocessed_sentence = preprocess_text(abstract_text)
  print(f"preprocessed_sentence : {preprocessed_sentence}")
  return preprocessed_sentence




---

**Preprocessing Words**

---



In [ ]:
stemmer = PorterStemmer()

In [ ]:
def remove_punctuations(words_list):
  filtered_words = []
  for word in words_list:
    if word.isalpha():
      filtered_words.append(word)

  return filtered_words

def get_words_with_nouns(words_list):
  filtered_words = []
  #With parts of speech assigned to each word
  words_list_pos = nltk.pos_tag(words_list)
  for (w, pos) in words_list_pos:
    if pos[:2] == 'NN':
      filtered_words.append(w)

  return filtered_words

def fix_typos(word_list):
    corrected_words = []
    for word in word_list:
        suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=3)
        if suggestions:
            corrected_words.append(suggestions[0].term)

    return corrected_words

def stem_words(words_list):
  stemmed_words = []
  for word in words_list:
    stemmed_words.append(stemmer.stem(word))

  return stemmed_words

stop_words = [
    'en', 'es', 'de', 'it', 'ca', 'pt', 'pl', 'da', 'ru', 'sv', 'sk', 'nl'
]

stop_word_list = list(set().union(*[get_stop_words(lang) for lang in stop_words]))

def remove_stop_words(words_list):

  words_list_no_stop_words = []

  for word in words_list:
    if word not in stop_word_list:
      words_list_no_stop_words.append(word)

  return words_list_no_stop_words

In [ ]:
def preprocess_words(input_sentence):
  print("In preprocess_words")
  words_tokenized = word_tokenize(input_sentence)
  print(f"Tokenized words: {words_tokenized}")

  words_list = remove_punctuations(words_tokenized)
  print(f"Words after removing punctuations : {words_list}")

  words_list_nouns = get_words_with_nouns(words_list)
  print(f"List after removing nouns : {words_list_nouns}")

  corrected_words_list = fix_typos(words_list_nouns)
  print(f"Corrected words List : {corrected_words_list}")

  # stemmed_words_list = stem_words(corrected_words_list)
  # print(f"Stemmed words : {stemmed_words_list}")

  # preprocessed_words_list = remove_stop_words( stemmed_words_list)
  preprocessed_words_list = remove_stop_words(corrected_words_list)
  print(f"Final Preprocessed Words List : {preprocessed_words_list}")

  return preprocessed_words_list



---

**Preprocess the data**

---



In [ ]:
def preprocess_data(articles, sample_size):
    print(f'Preprocessing the articles')
    articles_len = len(articles)
    preprocessed_sentences_list = []
    token_lists = []
    indices = []

    samples = np.random.choice(articles_len, sample_size)

    for i, idx in enumerate(samples):
        preprocessed_sentence = preprocess_sentence(articles[idx])
        token_list = preprocess_words(preprocessed_sentence)

        if token_list:
            indices.append(idx)
            preprocessed_sentences_list.append(preprocessed_sentence)
            token_lists.append(token_list)


    print(f"Preprocessed Sentences List : {preprocessed_sentences_list}")
    print(f"Token Lists : {token_lists}")
    print(f"Indices : {indices}")

    print('Done with Preprocessing the data')


    return preprocessed_sentences_list , token_lists, indices



In [ ]:
!pip install tensorflow
from sklearn.model_selection import train_test_split
import warnings

import keras
from keras.layers import Input, Dense
from keras.models import Model


In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
class Autoencoder:

    def __init__(self, latent_dim=32, activation='relu', epochs=200, batch_size=128):
        self.latent_dim = latent_dim
        self.activation = activation
        self.epochs = epochs
        self.batch_size = batch_size
        self.autoencoder_model = None
        self.encoder_model = None
        self.decoder_model = None
        self.history = None

    def compile_model(self, input_dim):
        print(f"In AutoEncoder Complile Model")
        input_vector = Input(shape=(input_dim,))
        encoded = Dense(self.latent_dim, activation=self.activation)(input_vector)
        decoded = Dense(input_dim, activation=self.activation)(encoded)
        self.autoencoder_model = Model(input_vector, decoded)
        self.encoder_model = Model(input_vector, encoded)
        encoded_input = Input(shape=(self.latent_dim,))
        decoder_layer = self.autoencoder_model.layers[-1]
        self.decoder_model = Model(encoded_input, decoder_layer(encoded_input))
        self.autoencoder_model.compile(optimizer='adam', loss=keras.losses.mean_squared_error)

    def fit_model(self, X):
        if not self.autoencoder_model:
            self.compile_model(X.shape[1])
        X_train, X_test = train_test_split(X)
        self.history = self.autoencoder_model.fit(X_train, X_train,
                                                  epochs=self.epochs,
                                                  batch_size=self.batch_size,
                                                  shuffle=True,
                                                  validation_data=(X_test, X_test),
                                                  verbose=0)




---

**Define Topic Modelling Class**

---



In [ ]:
import numpy as np
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import gensim
from gensim import corpora

In [ ]:
from sentence_transformers import SentenceTransformer
bert_model = SentenceTransformer('bert-base-nli-max-tokens')

In [ ]:
def get_lda_vectors(corpus, k, dictionary):
  lda_model = gensim.models.ldamodel.LdaModel(corpus, num_topics=k, id2word=dictionary,
                                                                passes=20)

  num_articles = len(corpus)
  vectors_lda = np.zeros((num_articles, k))

  for i in range(num_articles):
    for topic, prob in lda_model.get_document_topics(corpus[i]):
      vectors_lda[i, topic] = prob

  return vectors_lda



In [ ]:
def get_tf_idf_vectors(sentences):
  tf_idf_vectorizer = TfidfVectorizer()
  return tf_idf_vectorizer.fit_transform(sentences)


In [ ]:
def get_bert_vectors(sentences):
  encoded_sentences = bert_model.encode(sentences, show_progress_bar=True)
  return np.array(encoded_sentences)

In [ ]:
def get_lda_bert_vectors(corpus, k, dictionary, sentences):
  print("In get LDA_BERT Vectors Method")
  gamma = 15
  lda_vectors = get_lda_vectors(corpus, k, dictionary)
  bert_vectors = get_bert_vectors(sentences)
  lda_bert_vectors = np.c_[bert_vectors, lda_vectors*gamma]

  return lda_bert_vectors



In [ ]:
# define model object
class Topic_Model:
    def __init__(self, method, k= 10):
        self.k = k
        self.dictionary = None
        self.corpus = None
        self.gamma = 15
        self.method = method

        self.cluster_model = KMeans
        self.ldamodel = None
        self.vec = {}
        self.auto_encoder = Autoencoder()
        self.id = method + '_' + datetime.now().strftime("%Y_%m_%d_%H_%M_%S")


    def vectorize(self, sentences, token_lists, method):
        self.dictionary = corpora.Dictionary(token_lists)
        self.corpus = [self.dictionary.doc2bow(text) for text in token_lists]

        if method == 'TFIDF':
          tfidf_vectors = get_tf_idf_vectors(sentences)
          self.vec[method] = tfidf_vectors
          return tfidf_vectors

        elif method == 'LDA':
          lda_vectors = get_lda_vectors(self.corpus, self.k, self.dictionary)
          self.vec[method] = lda_vectors
          return lda_vectors

        elif method == 'BERT':
          bert_vectors = get_bert_vectors(sentences)
          self.vec[method] = bert_vectors
          return bert_vectors

        elif method == 'LDA_BERT':
          print(f"In Topic Model Vectorize Function")
          lda_bert_vectors = get_lda_bert_vectors(self.corpus, self.k, self.dictionary, sentences)
          self.vec[method] = lda_bert_vectors
          self.auto_encoder.fit_model(lda_bert_vectors)
          vectors = self.auto_encoder.encoder_model.predict(lda_bert_vectors)
          return vectors



    def fit(self, sentences, token_lists, method):
      self.dictionary = corpora.Dictionary(token_lists)
      self.corpus = [self.dictionary.doc2bow(text) for text in token_lists]

      clustering_method = KMeans
      if method == 'LDA':
        self.ldamodel = gensim.models.ldamodel.LdaModel(self.corpus, num_topics=self.k, id2word=self.dictionary,
                                                                passes=20)
      # else :
      #   self.cluster_model = clustering_method(self.k)
      #   self.vec[method] = self.vectorize(sentences, token_lists, method)
      #   self.cluster_model.fit(self.vec[method])
      self.cluster_model = clustering_method(self.k)
      self.vec[method] = self.vectorize(sentences, token_lists, method)
      self.cluster_model.fit(self.vec[method])


    def predict(self, sentences, token_lists, out_of_sample=None):
        out_of_sample = out_of_sample is not None

        if out_of_sample:
            corpus = [self.dictionary.doc2bow(text) for text in token_lists]
            if self.method != 'LDA':
                vec = self.vectorize(sentences, token_lists)
                print(vec)

        else:
            corpus = self.corpus
            vec = self.vec.get(self.method, None)

        if self.method == "LDA":
            lbs = np.array(list(map(lambda x: sorted(self.ldamodel.get_document_topics(x),
                                                     key=lambda x: x[1], reverse=True)[0][0],
                                    corpus)))
        else:
            lbs = self.cluster_model.predict(vec)

        return lbs

In [ ]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore', category=Warning)

import argparse

In [ ]:
def visualize(model):
    # if model.method == 'LDA':
    #     return
    reducer = umap.UMAP()
    vec_umap = reducer.fit_transform(model.vec[model.method])
    plot_proj(vec_umap, model.cluster_model.labels_)
    dr = '/content/sample_data/figures'.format(model.method, model.id)
    if not os.path.exists(dr):
        os.makedirs(dr)
    plt.savefig('/2D_vis')

    # Add labels outside the figure
    fig, ax = plt.subplots()
    for i, txt in enumerate(model.cluster_model.labels_):
        ax.annotate(txt, (vec_umap[i, 0], vec_umap[i, 1]), xytext=(5, 5),
                    textcoords='offset points', ha='right', va='bottom')

    plt.savefig('/2D_vis_with_labels')

In [ ]:
def main():
    # method = "LDA_BERT"
    sample_size = 1000
    ntopic = 12
    methods_list = ["BERT"]
    data = documents
    data = data.fillna('')
    rws = data['ABSTRACT']
    preprocessed_sentences, token_lists, indices = preprocess_data(rws, sample_size=sample_size)
    for method in methods_list:
      tm = Topic_Model(method = method, k = ntopic)
      print(f"Topic Model : {tm}")
      tm.fit(preprocessed_sentences, token_lists, method)
      print('Coherence:', get_coherence(tm, token_lists, 'c_v'))
      print('Silhouette Score:', get_silhouette(tm))
      visualize(tm)
      for i in range(tm.k):
        get_wordcloud(tm, token_lists, i)


    # tm = Topic_Model(method = method, k = ntopic)
    # print(f"Topic Model : {tm}")
    # tm.fit(preprocessed_sentences, token_lists, method)

    # print('Coherence:', get_coherence(tm, token_lists, 'c_v'))
    # print('Silhouette Score:', get_silhouette(tm))
    # visualize(tm)
    # for i in range(tm.k):
    #     get_wordcloud(tm, token_lists, i)

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

In [ ]:
main()

In [ ]:
def main():
    # method = "LDA_BERT"
    sample_size = 1000
    ntopic = 12
    methods_list = ["TFIDF"]
    data = documents
    data = data.fillna('')
    rws = data['ABSTRACT']
    preprocessed_sentences, token_lists, indices = preprocess_data(rws, sample_size=sample_size)
    for method in methods_list:
      tm = Topic_Model(method = method, k = ntopic)
      print(f"Topic Model : {tm}")
      tm.fit(preprocessed_sentences, token_lists, method)
      print('Coherence:', get_coherence(tm, token_lists, 'c_v'))
      print('Silhouette Score:', get_silhouette(tm))
      visualize(tm)
      for i in range(tm.k):
        get_wordcloud(tm, token_lists, i)

In [ ]:
main()

In [ ]:
def main():
    # method = "LDA_BERT"
    sample_size = 1000
    ntopic = 12
    methods_list = ["LDA_BERT"]
    data = documents
    data = data.fillna('')
    rws = data['ABSTRACT']
    preprocessed_sentences, token_lists, indices = preprocess_data(rws, sample_size=sample_size)
    for method in methods_list:
      tm = Topic_Model(method = method, k = ntopic)
      print(f"Topic Model : {tm}")
      tm.fit(preprocessed_sentences, token_lists, method)
      print('Coherence:', get_coherence(tm, token_lists, 'c_v'))
      print('Silhouette Score:', get_silhouette(tm))
      visualize(tm)
      for i in range(tm.k):
        get_wordcloud(tm, token_lists, i)

In [ ]:
main()

In [ ]:
# tm.fit(preprocessed_sentences, token_lists, method)

In [ ]:
# tm.vec['LDA_BERT']

In [ ]:
def main():
    # method = "LDA_BERT"
    sample_size = 1000
    ntopic = 12
    methods_list = ["LDA"]
    data = documents
    data = data.fillna('')
    rws = data['ABSTRACT']
    preprocessed_sentences, token_lists, indices = preprocess_data(rws, sample_size=sample_size)
    for method in methods_list:
      tm = Topic_Model(method = method, k = ntopic)
      print(f"Topic Model : {tm}")
      tm.fit(preprocessed_sentences, token_lists, method)
      print('Coherence:', get_coherence(tm, token_lists, 'c_v'))
      print('Silhouette Score:', get_silhouette(tm))
      visualize(tm)
      for i in range(tm.k):
        get_wordcloud(tm, token_lists, i)

In [ ]:
main()